In [1]:
!mkdir container

mkdir: cannot create directory ‘container’: File exists


In [2]:
%%writefile container/Dockerfile

# Build an image that can do training and inference in SageMaker

FROM continuumio/anaconda3:4.4.0

RUN conda install -c anaconda setuptools

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         libgl1-mesa-glx \
         wget \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN pip install --upgrade pip && \
    pip install janome==0.3.10 && \
    pip install tensorflow==1.14.0 --user && \
    pip install ml_metrics==0.1.4 && \
    pip install --upgrade scipy==1.1.0 && \
    pip uninstall --yes numpy && \
    pip install numpy==1.16.4 && \
    pip uninstall --yes gast && \
    pip install gast==0.2.2 && \
    pip install flask && \
    pip install -U gevent==1.4.0 --ignore-installed && \
    pip install gunicorn

RUN python -m nltk.downloader book

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

WORKDIR /opt/program

RUN git clone https://github.com/mack-the-psych/plimac3.git
RUN git clone https://github.com/mack-the-psych/vdok3.git

RUN echo "/opt/program/plimac3/Lib" > /opt/conda/lib/python3.6/site-packages/plimac-custom.pth
RUN echo "/opt/program/plimac3/Tools" >> /opt/conda/lib/python3.6/site-packages/plimac-custom.pth

RUN echo "/opt/program/vdok3/prep" > /opt/conda/lib/python3.6/site-packages/vdok3-custom.pth
RUN echo "/opt/program/vdok3/extract" >> /opt/conda/lib/python3.6/site-packages/vdok3-custom.pth
RUN echo "/opt/program/vdok3/process" >> /opt/conda/lib/python3.6/site-packages/vdok3-custom.pth
RUN echo "/opt/program/vdok3/reorganize" >> /opt/conda/lib/python3.6/site-packages/vdok3-custom.pth
RUN echo "/opt/program/vdok3/train" >> /opt/conda/lib/python3.6/site-packages/vdok3-custom.pth

WORKDIR /opt/program/plimac3/Resource/OANC
RUN python compile_shelve.py

COPY vdok3_sage /opt/program
WORKDIR /opt/program

Overwriting container/Dockerfile


In [3]:
%%time
%%sh

# The name of our algorithm
algorithm_name=sagemaker-vdok3-rf

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  43.52kB
Step 1/22 : FROM continuumio/anaconda3:4.4.0
 ---> 795ad88c47ff
Step 2/22 : RUN conda install -c anaconda setuptools
 ---> Using cache
 ---> fd4d3741b434
Step 3/22 : RUN apt-get -y update && apt-get install -y --no-install-recommends          libgl1-mesa-glx          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 0648247f740a
Step 4/22 : RUN pip install --upgrade pip &&     pip install janome==0.3.10 &&     pip install tensorflow==1.14.0 --user &&     pip install ml_metrics==0.1.4 &&     pip install --upgrade scipy==1.1.0 &&     pip uninstall --yes numpy &&     pip install numpy==1.16.4 &&     pip uninstall --yes gast &&     pip install gast==0.2.2 &&     pip install flask &&     pip install -U gevent==1.4.0 --ignore-installed &&     pip install gunicorn
 ---> Using cache
 ---> f9847961a041
Step 5/22 : RUN python -m nltk.downloader book
 ---> Using cache
 ---> af143b

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 5.92 ms, sys: 4.26 ms, total: 10.2 ms
Wall time: 8.3 s


In [4]:
# S3 prefix
prefix = 'vdok3_rf_trial'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [5]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [6]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [7]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-rf:latest'.format(account, region)

vdok3rf = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)

vdok3rf.fit(data_location)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-04 09:52:44 Starting - Starting the training job...
2020-10-04 09:52:46 Starting - Launching requested ML instances......
2020-10-04 09:53:52 Starting - Preparing the instances for training...
2020-10-04 09:54:36 Downloading - Downloading input data
2020-10-04 09:54:36 Training - Downloading the training image.........
2020-10-04 09:56:06 Training - Training image download completed. Training in progress../opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Starting the training.
Best parameters: {'max_depth': None, 'max_features': 1, 'min_samples_leaf': 1, 'n_estimators': 100}
Best score: 0.932
Training complete.

2020-10-04 09:57:23 Uploading - Uploading generated training model
2020-10-04 09:57:23 Completed - Training job completed
Training seconds: 173
Billable 

In [8]:
from sagemaker.predictor import csv_serializer
predictor = vdok3rf.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

In [9]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
# df_in = pd.read_csv('data/Key-Stem-Passage-Aggregate_plim-Def-PRE.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

Score_Class,Definition-Score
0,0
1,1
2,1
2,2



In [10]:
sess.delete_endpoint(predictor.endpoint)

In [11]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = vdok3rf.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: sagemaker-vdok3-rf-2020-10-04-09-52-44-237


In [12]:
transformer.transform(data_location + '/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', content_type='text/csv', split_type='Line')
transformer.wait()

s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out".format(transform_output_folder), '/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out')
with open('/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

.....................................2020-10-04T10:12:46.699:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
Starting the inference server with 4 workers.
2020/10/04 10:12:44 [crit] 11#0: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [04/Oct/2020:10:12:44 +0000] "GET /ping HTTP/1.1" 502 172 "-" "Go-http-client/1.1"
Starting the inference server with 4 workers.
2020/10/04 10:12:44 [crit] 11#0: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [04/Oct/2020:10:12:44 +0000] "GET /ping HTTP/1.1" 502 172 "-" 